# AB test 基本流程以及实例分析

本文结合多方资料，对AB test基本流程以及相关名次进行介绍，附加一个实例帮助理解

## AB test 基本流程

1. 分析现状，建立假设：分析业务，确定最高优先级的改进点，作出假设，提出优化建议。

2. 设定指标：设置主要指标来衡量版本的优劣；设置辅助指标来评估其他影响。

3. 设计与开发：设计优化版本的原型并完成开发。

4. 确定测试时长：确定测试进行的时长。

5. 确定分流方案：确定每个测试版本的分流比例及其他分流细节。

6. 采集并分析数据：收集实验数据，进行有效性和效果判断。

7. 给出结论：①确定发布新版本；②调整分流比例继续测试；③优化迭代方案重新开发，回到步骤1。

## 假设检验基本原理

从知乎上找到了一个很详细的图片

![](http://m.qpic.cn/psc?/V509KgjP2rVc1x3xXZE72dVD4k46B5pi/ruAMsa53pVQWN7FLK88i5jKjbPqA7QemKbYuKQcfzg.*bqdHsdpDplq6Xm7F1jg.LX6q8jqVY6vpIgQGiEeyepHXfJ5o*SMxkFtfZGTLU8o!/b&bo=0AJkAdACZAEBCS4!&rf=viewer_4)

### 基本概念

假设检验的基本思想是“小概率事件”原理, 小概率思想是指小概率事件在一次试验中基本上不会发生。

P值：H0发生的概率

通过P值判断结果是因为抽样误差还是本质差异所造成的

H<sub>0</sub>：无效假设，X=u<sub>0</sub>，无差异或者差异由误差引起

H<sub>1</sub>：备择假设，X<>u<sub>0</sub>，有差异

### 1. 零假设和备择假设

把没有充分理由不能轻易否定的命题作为零假设 H<sub>0</sub>，把没有把握不能轻易肯定的命题作为备择假设 H<sub>1

简单来说，把方案无效的假设作为 H<sub>0</sub>，把方案有效的假设作为 H<sub>1</sub>

我们希望的结果是实验有效果而不是误差导致的，也就是 H<sub>1</sub>推翻 H<sub>0</sub>

### 2. 假设检验的检验方向

备择假设含≠则为双尾；含<或>则为单尾，含<为左尾，含>为右尾

### 4. 确定检验类型以及检验样本

在判断用什么检验的时候，首要考虑的条件是样本量，其次是总体服从的分布

- T检验：用于样本含量较小(如n<60)，总体标准差σ未知的情况

- u(z)检验：若样本含量较大(如n>60)的情况，或者样本含量虽小，但总体标准差σ已知的情况

- f检验：检验方差齐性，要求样本均来自正态分布的总体。两样本方差比较的F检验是双侧检验

- 卡方检验：统计样本的实际观测值与理论推断值之间的偏离程度。两个率或两个构成比比较的卡方检验；多个率或多个构成比比较的卡方检验以及分类资料的相关分析

- 当样本容量小于30，且不满足总体近似服从正态分布，不能用Z检验和T检验

现在的很多软件简化了上述步骤，改为，若总体标准差已知（无论样本大小）都用Z检验；若总体标准差未知，都用T检验

### 5. 显著性水平α

α常取0.1、0.05、0.01

补充一下三者概念上的区别：置信区间，显著性水平，置信水平

- 置信区间：由样本统计量所构造的总体参数的估计区间（一个范围）
- 显著性水平α（犯错概率）：置信度100-α，α=0.05，置信度为95%，c1<=u<=c2,c1-c2就是置信区间
- 置信水平（置信度）：置信区间之内的概率值,(1-α)*100%

## 实例

案例数据是对web新旧页面的A/B测试结果，目标是判断新旧两版页面在用户的转化情况上是否有显著区别

### 1. 导入库

In [14]:
import numpy as np
import matplotlib as plt
import pandas as pd

### 2. 数据读取

In [15]:
df=pd.read_csv(r'F:\Data\ab_data9061\ab_data.csv')

### 3. 查看数据

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [17]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [18]:
df.shape

(294478, 5)

### 4. 处理user_id 列

In [19]:
len(df.user_id)

294478

In [20]:
len(df.user_id.unique())

290584

我们理解的user_id 应该是一个distinct的唯一值，所以需要对user_id进行处理

In [23]:
df.user_id.duplicated().sum()

3894

#### 重复值查看

In [24]:
df.loc[df.user_id.duplicated(keep=False)].sort_values(by='user_id')

,user_id,timestamp,group,landing_page,converted
230259,630052,2017-01-17 01:16:05.208766,treatment,new_page,0
213114,630052,2017-01-07 12:25:54.089486,treatment,old_page,1
22513,630126,2017-01-14 13:35:54.778695,treatment,old_page,0
251762,630126,2017-01-19 17:16:00.280440,treatment,new_page,0
183371,630137,2017-01-20 02:08:49.893878,control,old_page,0
...,...,...,...,...,...
142354,945703,2017-01-08 19:40:51.169351,control,new_page,0
186960,945797,2017-01-13 17:23:21.750962,control,old_page,0
40370,945797,2017-01-11 03:04:49.433736,control,new_page,1
165143,945971,2017-01-16 10:09:18.383183,control,old_page,0


实验组new_page应该保证group=treatment,对照组old_page保证group=control

In [25]:
df_new=df[(df['landing_page'] == 'new_page')&(df['group'] == 'treatment')]
df_old=df[(df['landing_page'] == 'old_page')&(df['group'] == 'control')]

In [26]:
df=pd.concat([df_new,df_old])

In [28]:
df.head()

,user_id,timestamp,group,landing_page,converted
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1


In [30]:
df.shape[0]

290585

In [29]:
len(df.user_id.unique())

290584

两者还是有一个数据的差距，继续清洗

In [32]:
df.user_id.duplicated().sum()

1

In [33]:
df[df.user_id.duplicated(keep=False)]

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


我们发现了一个user_id重复的数据，我们选取最近登录的时间

In [34]:
df.drop(df[df.user_id.duplicated(keep='first')].index,inplace=True)

#### 重看数据

In [35]:
df.user_id.duplicated().sum()

0

In [36]:
df.shape[0]

290584

In [37]:
df.user_id.nunique()

290584

In [38]:
df.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

数据清洗完毕

### 数据参数提取

用户总转化率

In [41]:
df.converted.mean()

0.12168942543292129

旧版用户转化率

In [42]:
df[df.group=='control'].converted.mean()

0.1203863045004612

新版用户转化率

In [43]:
df[df.group=='treatment'].converted.mean()

0.12299222352212512

新版用户占比

In [45]:
df[df.landing_page=="new_page"].shape[0]/df.shape[0]

0.5000619442226688

#### 结论

我们可以明显地看出，旧版用户和新版用户的转化率有明显的差距，所以我们需要验证这个差距是H<sub>0</sub>导致的还是H<sub>1</sub>导致的

### 假设检验

#### 1. 确定零假设和备择假设

旧页面的转化率为P<sub>0</sub>，新页面的转化率为P<sub>1</sub>

零假设H<sub>0</sub>：P1>=P2 即 P1-P2>=0

备择假设H<sub>1</sub>：P1<P2 即 P1-P2<0

#### 2. 检验方向

选择检验方向为左尾

#### 4. 确定检验类型及其统计量

首先查看两样本数据

In [47]:
df[df.landing_page=="new_page"].shape[0]

145310

In [48]:
df[df.landing_page=="old_page"].shape[0]

145274

两样本均为大样本，使用u(z)检验

#### 5. 确定显著性水平α

α取0.05

### 重要数值计算

旧版、新版用户转化数

In [67]:
con_old=df[(df.landing_page=="old_page")&(df.converted==1)].shape[0]
con_old

17489

In [69]:
con_new=df[(df.landing_page=="new_page")&(df.converted==1)].shape[0]
con_new

17872

旧版、新版用户数

In [56]:
n_old=df[df.landing_page=="old_page"].shape[0]
n_old

145274

In [57]:
n_new=df[df.landing_page=="new_page"].shape[0]
n_new

145310

旧版、新版用户转化率

In [60]:
p_old=df[df.landing_page=="old_page"]['converted'].mean()
p_old

0.1203863045004612

In [62]:
p_new=df[df.landing_page=="new_page"]['converted'].mean()
p_new

0.12299222352212512

旧版、新版用户标准差

In [64]:
std_old=df[df.landing_page=="old_page"].converted.std()
std_old

0.32541384592046235

In [65]:
std_new=df[df.landing_page=="new_page"].converted.std()
std_new

0.3284294121886301

### z值，p值计算

In [66]:
# 导入库
import statsmodels.stats.proportion as sp

因为P<sub>0</sub><P<sub>1</sub>，所以使用'smaller'

In [86]:
z_score, p_value = sp.proportions_ztest([con_old, con_new], [n_old, n_new], alternative='smaller')

In [87]:
z_score

-2.1484056695589

In [88]:
p_value

0.015840771394875417

我们判断的p值p=0.01584,p<α,所以我们认为H<sub>0</sub>发生的概率小于α，根据“小概率事件”原理，我们拒绝H<sub>0</sub>，接受H<sub>1</sub>

### 报告

- 旧版页面平均转化用户数为0.120个，标准差为0.325
- 新版页面平均转化用户数为0.123个，标准差为0.328
- 独立双样本z=-2.15，p=0.01584(α=0.05)，单尾检验（左尾），拒绝零假设。

## 结论

- 在这个实验中，我们的目的是找出登录页是否对转换率有显著的影响。
- 因此，我们的无效假设是旧登录页的转换率与新页相同甚至更高，而我们的另一个假设是旧登录页的转换率低于新页。
- 我们得出结论，即登陆页面对转换率有显著影响